# Running Molecular Dynamics Simulations in OpenMM

## Overview
**Molecular Dynamics Simulation**: simulation of molecules through time via calculations of forces on atoms and their positions based on those forces. The output is a trajectory.

## Steps for running a simulation
1. **Initialization** -- setting up imports, variables, and other system parameters to prepare for the simulation
2. **Minimization** -- calculating the energies of moving molecules slightly to obtain a local "minimum" in energy and eliminate “bad” interatomic contacts
3. **Equilibration** -- a brief MD simulation that prepares our system to our target temperature and desired equilibrium values
4. **Production** -- run the equilibration
5. **Analysis** -- after you have collected your data from the production run, you must analyze the trajectory to draw conclusions

## Our First Simulations
We will now use [OpenMM](https://openmm.org/documentation) to do a molecular dynamics simulation of the ethane and butane molecules we prepared. It’s important to note at this point that molecular dynamics simulations can be performed using a number of softwares. However, we will be running a simulation with a program called OpenMM. OpenMM has the advantage of being scriptable with Python.

---

### Initialization
Once you have OpenMM, we can use it to simulate our molecules. Start in your notebook with imports. Here are the python libraries you will need to run simulations with OpenMM.

In [1]:
import openmm as mm
from openmm import app, unit

First, we need to read in our structure and our force field. We have to tell the simulation our initial coordinates and the force field we will use. To do this, we use the PDB file we have and the force field file we prepared.

In [2]:
pdb = app.PDBFile('data/ethane.pdb')
forcefield = app.ForceField('data/ethane.gaff2.xml')

#### QUESTION
What are we doing with these two commands above?
*(Hint: let's check out some [documentation](http://docs.openmm.org/latest/userguide/application/02_running_sims.html))*

#### RESPONSE
The first command loads the `.pdb` file (atomic position data) and returns a PDBFile object.
The second command loads the `.xml` file (bond and force field parameters) and returns a ForceField object.

#### QUESTION
What lies in the contents of the XML file being read in `forcefield`?

#### RESPONSE
In the `Residue` tag with `Atom` tags. Each `Atom` tag has a unique `name` for the atom and `type` attribute specifying the type of atom. The `type` values were defined in the `AtomTypes` tag. 


There are `HarmonicBondForce`, `HarmonicAngleForce`, `PeriodicTorsionForce`, `NonbondedForce` tags with the MM parameters given in specific attributes.

For example, the force constant is found in `HarmonicBondForce.Bond[k]` (psuedo code where [] are attributes). The bond angles can be found in `HarmonicAngleForce.Angle[hc]`

Next, we set up the system for our MD simulation. With the following command, we use the `pdb` variable to create a system. The other arguments to the function say that we are not using a cut-off and that we want to constrain bonds with hydrogens (this allows us to use a larger timestep).

In [3]:
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

Our simulation will be in vacuum at a temperature of 298.15 K. The Langevin integrator is what is called a stochastic integrator. This means that it mimics jostling of air or solvent through random forces. We are using a 5.0 picosecond coupling constant, which is something which controls how often the integrator adds jostling motions.

In [4]:
integrator = mm.LangevinIntegrator(298.15*unit.kelvin, 5.0/unit.picoseconds, 2.0*unit.femtoseconds)
integrator.setConstraintTolerance(1e-5)

#### QUESTION
What is the `integrator` applying to our simulation?

#### RESPONSE
The `LangevinIntegrator` is applying random force at a constant interval to our system (ethane molecule). A random force (collisions/interactions) is applied due to the random nature of particles in air/solvent. The Langevin equation is commonly used to simulate the random (Brownian) motion of an atom/particles/molecules and diffusion.

The `integrator` is basically simulating brownian motion to our system when equilibrating and simulating.

Finally, we initialize the simulation by adding all of the pieces we have prepared:

In [6]:
platform = mm.Platform.getPlatformByName('Reference')
simulation = app.Simulation(pdb.topology, system, integrator, platform) # ENTER THE THREE PIECES OF OUR SIMULATION HERE (hmm... I wonder where I can find this)
simulation.context.setPositions(pdb.positions)

---

### Minimization
Now, we start calculating energies. First we do an energy minimization. We start in this code block by printing the energy before minimization, doing 100 steps of an energy minimization, then printing the new energy. You should see that the energy decreases:

In [7]:
print('Minimizing...')

st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy before minimization is {st.getPotentialEnergy()}")

simulation.minimizeEnergy(maxIterations=100)

st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy after minimization is {st.getPotentialEnergy()}")

Minimizing...
Potential energy before minimization is 4.467818271065188 kJ/mol
Potential energy after minimization is 4.389967724953272 kJ/mol


You can’t see it from this code, but the atom positions have changed slightly to cause this change in energy.

#### QUESTION
What is an energy minimization? Why do we want to perform a local energy minimization at the start of our simulation?

#### RESPONSE
Energy minimization is the process where a algorithm modifies the bond lengths, angles, dihedral angles, etc. such that the system is at a local minimum. We perform a local energy minimization since the real system we are modeling contains molecules near the local energy. This relates to the partition function and that there is higher statistically likelihood that molecules will occupy lower energy levels than higher energy levels.

---

### Equilibration
Next, we run an equilibration. The purpose of this equilibration is to get our system to our target temperature and to get the system equilibrated and ready for our production run.

In [8]:
from sys import stdout

print('Equilibrating...')

simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True, 
    potentialEnergy=True, temperature=True, separator=','))
simulation.context.setVelocitiesToTemperature(150.0*unit.kelvin)
simulation.step(2500)

Equilibrating...
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
100,11.627046749243728,266.17155539221443
200,15.565851748622029,319.2551752867935
300,22.879096125985395,138.6435326857513
400,14.938996506083694,277.00695877459117
500,23.837397866789487,335.55105762639863
600,32.65592024980232,329.5966330756701
700,16.098470870943167,372.5751518581214
800,18.99642526534693,341.3503998299196
900,21.000998938686585,483.2920328519226
1000,18.290478808559723,524.67895586422
1100,14.884062424160586,661.165525162624
1200,31.227162538521,341.3838189279901
1300,33.512466004300485,417.9866781180548
1400,28.724336916068513,381.86874201740756
1500,22.810923522806736,288.40826088883614
1600,21.27593003584123,418.64443410987167
1700,18.37367577739829,410.70392592715694
1800,18.271811274852602,413.5154166597259
1900,22.43649967854203,274.732914085288
2000,9.373123283526539,195.2661186050685
2100,18.178613607452583,282.1483774850332
2200,13.029571291085913,205.493188800625
2300,15.537968972598

The first command in this section sets up what information OpenMM will report to us as the simulation runs. We are asking for the potential energy, and temperature every 100 timesteps. By putting `stdout` as the first argument, we are telling the program to just print the information. Note that `stdout` comes from the built in Python module `sys`. If we wanted the information in a file instead, you would put the file name.

The second command sets the initial velocities of the system to a temperature equivalent of 150 K. Then, we integrate for 2,500 steps to allow the system to equilibrate.

---

### Production
This next block of code is a longer block of simulation called the ‘production run’. We’re also added a timer to this code so we can see how long it took our simulation to run.

In [13]:
import time as time

print('Running Production...')

# Begin timer
tinit=time.time()

# Clear simulation reporters
simulation.reporters.clear()

# Reinitialize simulation reporters. We do this because we want different information printed from the production run than the equilibration run.
# output basic simulation information below every 250000 steps - (which is equal to 2 fs(250,000) = 500,000 fs = 500 ps)
simulation.reporters.append(app.StateDataReporter(stdout, 250000, 
    step=True, time=True, potentialEnergy=True, temperature=True, 
    speed=True, separator=','))

# write out a trajectory (i.e., coordinates vs. time) to a DCD
# file every 100 steps - 0.2 ps
simulation.reporters.append(app.DCDReporter('ethane_sim.dcd', 100))

# run the simulation for 1.0x10^7 steps - 20 ns
simulation.step(1000000)

# End timer
tfinal=time.time()
print('Done!')
print('Time required for simulation:', tfinal-tinit, 'seconds')

Running Production...
#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
16750000,33500.00000590734,15.405585832689175,137.3621668026364,0
17000000,34000.000006009206,14.847295753904314,343.1438669922197,7.66e+03
17250000,34500.00000611107,15.621350666306537,237.36134438090653,7.57e+03
17500000,35000.00000621293,13.06896119784729,40.819175237619106,7.46e+03
Done!
Time required for simulation: 22.57840609550476 seconds


After executing this cell in your notebook, you should see an output which gives you the step number, simulation time, potential energy, temperature, and “speed” for steps in the simulation. The spacing of theses is set in the `simulation.reporters` step where we indicated we wanted information printed every 250,000 timesteps.

The “speed” is reported in “ns/day” or “nanoseconds/day”. This is a commonly used unit to report how quickly simulations run. It tells you how much simulation time in nanoseconds would pass for 24 hours of computation time. For example, if a simulation is running at 2 ns/day, enough timesteps would be calculated in one day to make 2 nanoseconds of simulation time. If we were using our 2 fs timestep, this would mean that the computer calculated 1,000,000 timesteps over 24 hours.

---

## Your Turn
Make a copy of the code you wrote to run your ethane simulation and modify it to:
1. Read in the files `butane.gaff2.xml` and `butane.pdb`
2. Carry out a 10 ps MD simulation to bring the butane molecule to an equilibrium temperature of 298 K in which output is printed every 0.5 ps (Leave the minimization portion beforehand unchanged.)
3. Carry out a 40 ns MD simulation at 298 K in which output is printed every 1 ns and structures are (still) saved every 0.2 ps into a file called `butane_sim.dcd`.

In [15]:
### your code here!!!
# INIT
pdb = app.PDBFile('data/butane.pdb')
forcefield = app.ForceField('data/butane.gaff2.xml')

system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

integrator = mm.LangevinIntegrator(298.15*unit.kelvin, 5.0/unit.picoseconds, 2.0*unit.femtoseconds)
integrator.setConstraintTolerance(1e-5)

platform = mm.Platform.getPlatformByName('Reference')
simulation = app.Simulation(pdb.topology, system, integrator, platform) # ENTER THE THREE PIECES OF OUR SIMULATION HERE (hmm... I wonder where I can find this)
simulation.context.setPositions(pdb.positions)

# Minimization
print('Minimizing...')
st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy before minimization is {st.getPotentialEnergy()}")
simulation.minimizeEnergy(maxIterations=100)
st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy after minimization is {st.getPotentialEnergy()}")

# Equilibration
print('Equilibrating...')
simulation.reporters.append(app.StateDataReporter(stdout, 5000, step=True, 
    potentialEnergy=True, temperature=True, separator=','))
simulation.context.setVelocitiesToTemperature(150.0*unit.kelvin)
simulation.step(2500)

# Production
print('Running Production...')
tinit=time.time()  # Begin timer
simulation.reporters.clear()  # Clear simulation reporters
# Reinitialize simulation reporters.
# output basic simulation information below every 500000 steps - (which is equal to 2 fs(500,000) = 1000,000 fs = 1000 ps = 1ns)
simulation.reporters.append(app.StateDataReporter(stdout, 500000, 
    step=True, time=True, potentialEnergy=True, temperature=True, 
    speed=True, separator=','))
# write out a trajectory (i.e., coordinates vs. time) to a DCD
# file every 100 steps - 0.2 ps
simulation.reporters.append(app.DCDReporter('butane_sim.dcd', 100))
# run the simulation for 2.0x10^7 steps - 40 ns
simulation.step(20000000)
# End timer
tfinal=time.time()
print('Done!')
print('Time required for simulation:', tfinal-tinit, 'seconds')

Minimizing...
Potential energy before minimization is 5.797528692127868 kJ/mol
Potential energy after minimization is 5.295784483017471 kJ/mol
Equilibrating...
Running Production...
#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
500000,999.9999999901769,53.83502643035473,427.0675448184399,0
1000000,1999.9999999665301,57.32709470388116,287.68589076707264,5.47e+03
1500000,2999.9999999428833,36.71826492400706,329.30299473097733,5.34e+03
2000000,3999.9999999192364,49.13881470499726,217.95380800400795,5.23e+03
2500000,5000.000000101135,38.875179555130494,383.54963414997997,5.24e+03
3000000,6000.000000304862,52.31869710109108,281.4682790492577,5.29e+03
3500000,7000.0000005085885,43.288606191226194,324.19629707666064,5.32e+03
4000000,8000.000000712315,39.17146249307399,291.281383447507,5.36e+03
4500000,9000.000000916041,49.59154419500255,454.012361558532,5.37e+03
5000000,10000.000001119768,37.25176644182051,405.58025922219326,5.38e+03
5500000,11000.0000013

---

## Analysis
Now that we’ve performed our computer experiment, it is time to analyze the data we have collected. The main type of data you have collected through this simulation is information on atom positions, or the system trajectory.

As part of our production simulation, we set up a reporter to record atomic positions. The code below shows that code from your previous script, **you do not need to execute it.**

```Python
simulation.reporters.append(app.DCDReporter('ethane_sim.dcd', 100))
```

This reporter saved the atomic positions for us every 100 timesteps in a file called `ethane_sim.dcd`. The DCD file format is a binary file (instead of being a text file), so you cannot open it and look at it. However, we will be using certain libraries to analyze and view the file’s contents. If you’ve run your simulation, you should have the file `ethane_sim.dcd` in the same folder as your Jupyter notebook.

First, we will need to make sure we have a few more Python libraries installed which can help us with analysis. We will use a library called **nglview** to visualize the trajectory, and a library called **MDTraj** to analyze the trajectory. Before opening a new notebook for analysis, you may need to install **nglview** and **MDTraj**.

Type the following in your *terminal* to install **nglview** and **MDTraj**:

In [ ]:
$ conda install -c conda-forge mdtraj nglview

In [16]:
import mdtraj as md

traj = md.load('ethane_sim.dcd', top='data/ethane.pdb')

The command above reads all of the atomic positions from `ethane_sim.dcd` and keeps track of atom connectivity (topology) which was given in the PDB file. Next, visualize the trajectory using nglview. Nglview has a special function `show_mdtraj` that we can use with our trajectory because it was in a specific format from the MDTraj library.

In [17]:
import nglview as ngl

visualize = ngl.show_mdtraj(traj)
visualize

NGLWidget(max_frame=9999)

This should show you something that looks sort of like a movie of your ethane molecule. These are the atomic positions calculated by OpenMM during the molecular dynamics run. We can now analyze the positions to find out some things about our molecule.

We will use another OpenMM command to pull out our bonds and atoms for analysis

In [18]:
atoms, bonds = traj.topology.to_dataframe()
atoms

,serial,name,element,resSeq,resName,chainID,segmentID
0,1,C1,C,1,ETH,0,
1,2,H11,H,1,ETH,0,
2,3,H12,H,1,ETH,0,
3,4,H13,H,1,ETH,0,
4,5,C2,C,1,ETH,0,
5,6,H21,H,1,ETH,0,
6,7,H22,H,1,ETH,0,
7,8,H23,H,1,ETH,0,


#### QUESTION
Describe the table that `atoms` produced.

#### RESPONSE
Your response here :D

### Analyzing the C-C bond length
Let’s look at what C-C bond lengths our ethane molecule had during the simulation. Before we can measure the bond lengths, we have to decide which atoms from our molecule define the bond angle. You should see a table output from running `atoms`.

We have to pick the atom indices for the C-C bond. An atom’s index is the left-most value in the table above. For our torsion, we’ll measure `C1-C2` the indices for these are `0` and `4`. We use the function `compute_distances` in the MDTraj library to measure the distance between these atoms.

In [ ]:
bond_indices = [0, 4] # atoms to define the bond length
bond_length = md.compute_distances(traj, [bond_indices])

We now have the measurement for this torsion angle in radians for each recorded timestep of the trajectory saved in the array `bond_length`. One way we can examine this data is by plotting it as a histogram using the Python library `matplotlib`.

In [ ]:
import matplotlib.pyplot as plt

bondcounts, binedges, otherstuff = plt.hist(bond_length, bins=120)
plt.title('C-C bond length histogram')
plt.xlabel('Bond length (nm)')
plt.ylabel('Counts')
plt.show()

## Your Turn
A torsion is made up of four atoms which are bonded to each other. Analyze the torsion angle associated with the atoms `H11-C1-C2-H21` for your trajectory. Instead of using the function `compute_distance`, use `compute_dihedrals`. Create a histogram plot of the torsion angles.

First, we need to pick the atom indices of our torsion angle and use the `compute_dihedrals` function to calculate the dihedrals.

In [ ]:
phi_indices = [] # add atoms to define the torsion angle
phi = # use the compute_dihedrals function

print(phi)

We now have the measurement for this torsion angle in radians for each recorded timestep of the trajectory.

Next, we can examine this data by plotting it as a histogram using the Python library `matplotlib`.

In [ ]:
import matplotlib.pyplot as plt

# create a histogram with 90 bins

Credit for this exercise goes to [MolSSI](https://education.molssi.org/mm-tools/02-md-alkanes/index.html).